In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# 1️⃣ Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers  # <-- This line is critical!

# 2️⃣ Load the data
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# 3️⃣ Convert labels: 'ham' -> 0, 'spam' -> 1
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# 4️⃣ Vectorize text
max_vocab_size = 10000
max_sequence_length = 100

vectorizer = layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# 5️⃣ Adapt vectorizer on training data
vectorizer.adapt(train_data['message'])

# 6️⃣ Vectorize text
X_train = vectorizer(train_data['message'])
X_test = vectorizer(test_data['message'])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

# 7️⃣ Build the model
model = tf.keras.Sequential([
    layers.Embedding(input_dim=max_vocab_size, output_dim=32),
    layers.GlobalAveragePooling1D(),
    layers.Dense(24, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 8️⃣ Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 9️⃣ Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

# 🔟 Define predict_message function
def predict_message(msg):
    vectorized_msg = vectorizer([msg])
    prediction = model.predict(vectorized_msg)[0][0]
    label = "spam" if prediction >= 0.5 else "ham"
    return [float(prediction), label]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, e.g. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Vectorize the input message
    vectorized_text = vectorizer([pred_text])

    # Make a prediction
    prediction = model.predict(vectorized_text, verbose=0)[0][0]  # get the scalar probability

    # Determine label
    label = 'spam' if prediction >= 0.5 else 'ham'

    # Return result in required format
    return [float(prediction), label]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
